In [6]:
import time
from openai import OpenAI
import os
from dotenv import load_dotenv
from typing import List, Dict, Any
from io import BytesIO
from asyncio import StreamReader

In [7]:
load_dotenv()
client = OpenAI(api_key=os.getenv('Q_API_KEY'))

In [28]:
from typing import List, Dict, Any
from io import BytesIO
from asyncio import StreamReader

# Função para gerar mensagens
def generate_message(question: Dict[str, Any], answer_text: str) -> List[Dict[str, str]]:
    return [
        {
            "role": "system",
            "content": f"Você é um professor criterioso de {question['disciplineName']} corrigindo uma questão discursiva de uma prova e que busca ajudar o seu aluno a melhorar.",
        },
        {
            "role": "user",
            "content": (
                f"Com base em provas e concursos passados, simule uma questão discursiva de concurso público (pode ser real) sobre {question['disciplineName']}\n"
                f"- Crie uma resposta ideal de exemplo\n"
                f"- Crie instruções para a correção da pergunta\n"
                f"- Coloque a pergunta dentro da tag <question>, o exemplo de resposta dentro da tag <example>"
            ),
        },
        {
            "role": "assistant",
            "content": (
                f"Pergunta: <question>{question['statement']}</question>"
                f"Resposta Exemplo: <example>{question['mirror']}</example>"
            ),
        },
        {
            "role": "user",
            "content": (
                "Com base nas instruções discutidas e nos itens listados abaixo, preciso que você avalie a minha resposta como um professor.\n"
                "- Verifique se a resposta é ofensiva ou irrelevante em relação à pergunta. Caso seja ofensiva ou irrelevante, responda com algum texto como o exemplo \"Não foi possível avaliar sua resposta, pois não está relacionada com a pergunta proposta\" e não continue a avaliação.\n"
                "- Se a minha resposta for a própria pergunta ou muito parecida, responda com algum texto parecido com o exemplo \"Não é possível avaliar a sua resposta, pois você simplesmente repetiu a pergunta original. Lembre-se de que em uma questão discursiva, é fundamental desenvolver uma argumentação própria e não apenas repetir o enunciado.\" e não continue a avaliação.\n"
                "- Seja gentil e discorra de forma clara e educativa os pontos e fale na primeira pessoa como meu professor.\n"
                "- Considere a minha resposta somente o conteúdo de <userAnswer> que virá ao final das instruções.\n"
                "- Separe sua avaliação em 3 partes:\n"
                "  -- 1) Conceito Geral (Nessa parte, avalie se todos os conceitos e informações esperadas para a resposta foram abordados, use o conteúdo de <example> como comparação, mas não se limite a ele).\n"
                "  -- 2) Coerência e Coesão (Nessa parte, verifique a coerência e coesão da minha resposta, como os conceitos foram apresentados e o que pode melhorar).\n"
                "  -- 3) Dicas (Nessa parte, me dê dicas de como melhorar a minha resposta, quais conceitos deveria ter abordado, como poderia ter redigido de uma forma melhor e outros pontos que considerar relevante.)\n"
                "- Não utilize tags HTML.\n"
                "- Não utilize de tags Markdown como **.\n"
                f"Minha resposta: <userAnswer>{answer_text}</userAnswer>"
            ),
        }
    ]

# Função principal para gerar feedback
async def generate_grade_stream(
    env: Dict[str, str],
    question: Dict[str, Any],
    content: str,
    use_strategy: str = "default",
    author_id: int = 0,
):
    openai.api_key = env["OPENAI_API_KEY"]
    
    # Configuração da estratégia
    strategies = {
        "default": {
            "fn": generate_message,
            "model": "gpt-4"
        }
    }
    
    strategy = strategies[use_strategy]
    messages = strategy["fn"](question, content)
    model = strategy["model"]
    
    # Gerar resposta da API
    response = openai.ChatCompletion.create(
        model=model,
        temperature=0.75,
        top_p=0.75,
        max_tokens=1500,
        messages=messages,
        stream=True,
    )
    
    feedback_content = ""
    try:
        async for part in response:
            delta_content = part["choices"][0]["delta"].get("content", "")
            if delta_content:
                feedback_content += delta_content
                print(delta_content, end="")
        
        # Salvar o feedback
        await touch_answer(env, question["id"], content, author_id, feedback_content)
    except Exception as e:
        print(f"Error during streaming: {e}")

# Função fictícia para salvar a resposta
async def touch_answer(env: Dict[str, str], question_id: int, content: str, author_id: int, feedback: str):
    # Simulação de gravação em banco de dados ou outra operação
    print(f"Saving feedback for question {question_id}: {feedback}")

In [29]:
# Run Step by Step
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

for step in run_steps.data[::-1]:
    print('======= Step >', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('```')
            print(tool_call.code_interpreter.input)
            print('```')
            if tool_call.code_interpreter.outputs:
                if tool_call.code_interpreter.outputs[0].type == 'logs':
                    print('Result')
                    print(tool_call.code_interpreter.outputs[0].logs)
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == 'text':
            print(message.content[0].text.value)

======= Step > message_creation
```python
import math

def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)

a = 4
b = 6
c = 8

lcm_ab = lcm(a, b)
result = lcm(lcm_ab, c)
result
```


In [30]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role='user',
  content=f"""Você deve executar o seguinte código Python: {python_code}. Não faça modificações ou adições ao código, e não forneça explicações ou saídas detalhadas. Apenas execute o código e retorne o resultado da execução."""
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
if run.status == 'completed':
  mensagens = client.beta.threads.messages.list(
      thread_id=thread.id
  )
else:
  print('Erro', run.status)

execution_result = mensagens.data[0].content[0].text.value
print(execution_result)


24


In [31]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role='user',
  content=f'''Compare o seguinte resultado: {execution_result} obtido pelo código Python: {python_code} com o gabarito oficial: {official_answer_key}. O resultado fornecido pelo código Python é suficiente para alcançar a resposta do gabarito oficial? Responda apenas com "SIM" ou "NÃO".'''
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
if run.status == 'completed':
  mensagens = client.beta.threads.messages.list(
      thread_id=thread.id
  )
else:
  print('Erro', run.status)

execution_result = mensagens.data[0].content[0].text.value
print(execution_result)


SIM


In [32]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role='user',
  content=f'''
  Você é um especialista em Matemática sua tarefa é fornecer soluções detalhadas e explicações claras, bem documentadas. Seu papel é guiar o aluno para uma compreensão profunda do tema abordado e da resposta correta, através de explicações detalhadas e passo a passo. Você interage com o aluno de forma educada e gentil, evitando repetir o enunciado da questão. Suas respostas são elaboradas para facilitar o entendimento integral do assunto em discussão. Com sua experiência, você fornece uma aprendizagem que verdadeiramente dialoga com as necessidades do aluno.
  Escreva um gabarito comentado sobre seguinte questão: {question} seguindo os passos abaixo:
  - Explique o tema da questão, como ela aborda o tema e os conhecimentos necessários para resolvê-la.
  - Justifique a resposta correta.
  - Comece sua mensagem deixando claro qual é a resposta correta e depois desenvolva e explique a questão.
  - Mostre o passo a passo de como resolver matemáticamente o problema.'''
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
if run.status == 'completed':
  mensagens = client.beta.threads.messages.list(
      thread_id=thread.id
  )
else:
  print('Erro', run.status)

execution_result = mensagens.data[0].content[0].text.value
print(execution_result)

### Resposta correta: 24 horas

#### Tema da Questão:
A questão apresentada requer o conhecimento de mínimo múltiplo comum (MMC) e frações. Especificamente, ela lida com intervalos de tempo e a necessidade de encontrar quando três intervalos coincidem novamente, o que é um típico problema em que se aplica o conceito de MMC. 

#### Justificativa da Resposta Correta:
Para saber em qual momento Gustavo tomará os três remédios juntos novamente, devemos encontrar o menor intervalo de tempo que seja múltiplo dos intervalos de tempo de cada um dos remédios. Este é um problema clássico de MMC, que busca o menor número positivo que é divisível por todos os números dados.

#### Passo a Passo da Solução:

1. **Identificação dos intervalos:**
    - Remédio 1: a cada 4 horas
    - Remédio 2: a cada 6 horas
    - Remédio 3: a cada 8 horas

2. **Aplicação do conceito de MMC:**
    - Precisamos encontrar o MMC dos números 4, 6 e 8.

3. **Decomposição dos números em fatores primos:**
    - Para calcula

In [34]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "Você irá receber um código e quero que você converta o código  que pode estar em LaTeX ou Markdown, em HTML, seguindo as diretrizes abaixo: 1. Formatação Geral: Use a tag `<p>` para cada parágrafo. Para títulos, converta qualquer tag `<h1>`, `<h2>`, etc., em `<p>` com estilo de negrito e tamanho de fonte ajustado para manter a hierarquia visual. 2. Destaque de Conceitos: Coloque conceitos importantes em negrito usando `<span style='font-weight: bold;'>`. Use cores para destacar os elementos: Fórmulas matemáticas: cor laranja escuro (`darkorange`). Conceitos e termos importantes: cor azul escuro (`darkblue`). 3. Conversão de Fórmulas Matemáticas: Para fórmulas matemáticas, utilize a tag `<math>` e subtags apropriadas (`<mi>`, `<mo>`, `<msup>`, `<mrow>`, etc.). Assegure-se de que as fórmulas estejam claras e bem formatadas, considerando quebras de linha, alinhamento e legibilidade. 4. Conversão de Elementos Específicos: Converta listas, tabelas e códigos em suas respectivas tags HTML (`<ul>`, `<ol>`, `<table>`, `<code>`, etc.), mantendo a formatação visual consistente. Para links e imagens, converta as tags de Markdown (`[texto](url)` e `![alt](url)`) para as tags HTML `<a>` e `<img>`. 5. Exemplo de Conversão LaTeX para HTML: Fórmula Simples. Entrada LaTeX: `E = mc^2`. Saída HTML: `<p><span style='font-weight: bold; color: darkorange;'><math><mrow><mi>E</mi><mo>=</mo><mi>m</mi><msup><mi>c</mi><mn>2</mn></msup><f/mrow></math></span></p>"},
    {"role": "user", "content": execution_result}
  ]
)

print(response.choices[0].message.content)

```html
<p style="font-size: 20px; font-weight: bold;">Resposta correta: 24 horas</p>

<p style="font-size: 18px; font-weight: bold;">Tema da Questão:</p>
<p>A questão apresentada requer o conhecimento de <span style='font-weight: bold; color: darkblue;'>mínimo múltiplo comum (MMC)</span> e frações. Especificamente, ela lida com intervalos de tempo e a necessidade de encontrar quando três intervalos coincidem novamente, o que é um típico problema em que se aplica o conceito de MMC.</p>

<p style="font-size: 18px; font-weight: bold;">Justificativa da Resposta Correta:</p>
<p>Para saber em qual momento Gustavo tomará os três remédios juntos novamente, devemos encontrar o menor intervalo de tempo que seja múltiplo dos intervalos de tempo de cada um dos remédios. Este é um problema clássico de MMC, que busca o menor número positivo que é divisível por todos os números dados.</p>

<p style="font-size: 18px; font-weight: bold;">Passo a Passo da Solução:</p>

<p><span style='font-weight: bold